<a href="https://colab.research.google.com/github/lasource18/CQF_Labs/blob/main/FED_Meeting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install OpenAI Whisper Python Package

In [ ]:
!pip install git+https://github.com/openai/whisper.git -q

In [ ]:
!nvidia-smi -L

Whisper transcribes audio speech to text

In [ ]:
!pip install pytube -q

In [ ]:
import whisper
from pytube import YouTube

In [ ]:
# model types: tiny, base, small, medium, large
model = whisper.load_model('base')

In [ ]:
#yt_vid_url = 'https://youtube.com/watch?v=NT2H9iyd-ms'
yt_vid_url = 'https://www.youtube.com/watch?v=UYnc6bsgkJQ'
yt_vid = YouTube(yt_vid_url)

In [ ]:
yt_vid.title

In [ ]:
yt_vid.streams

In [ ]:
dir(yt_vid)

In [ ]:
for stream in yt_vid.streams:
  print(stream)

In [ ]:
streams = yt_vid.streams.filter(only_audio=True)
streams

In [ ]:
stream = streams.first()
stream

In [ ]:
stream.download(filename='fed_meeting.mp4')

We do some additional processing of the audio file.
We choose to ignore any additional sound and speech after Jeromme Powell speaks. So we'll use ffmpeg for this.

The command will start the audio at the 378 seconds mark when he starts to speak and continue for 2715 seconds and chop off the rest of the audio and save the results in a new file cal fed_meeting_trimmed.mp4

In [ ]:
#!ffmpeg -ss 378 -i fed_meeting.mp4 -t 2715 fed_meeting_trimmed.mp4

In [ ]:
import datetime

# save a timestamp before transcription
t1 = datetime.datetime.now()
print('started at', t1)

# do the transcription
output = model.transcribe('fed_meeting.mp4')

# show time elapsed after transcription is complete
t2 = datetime.datetime.now()
print('ended at', t2)
print('time elapsed', t2 - t1)

In [ ]:
output['segments']

In [ ]:
import pandas as pd

In [ ]:
spy = pd.read_csv('SPY.csv')
#spy.set_index('date', inplace=True)

In [ ]:
spy

In [ ]:
for segment in output['segments']:
  second = int(segment['start'])
  second = second - (second % 5)
  spy.loc[second / 5, 'text'] = segment['text']

In [ ]:
spy = spy[:574]

In [ ]:
spy['percent'] = ((spy['close'] - spy['open']) / spy['open']) * 100
spy

In [ ]:
big_upmoves = spy[spy.percent > .05]
big_upmoves

In [ ]:
!pip install mplfinance -q
import mplfinance as mpf

In [ ]:
df = spy.copy()
df.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Text', 'Percent']
#print(df)
df.set_index('Date', inplace=True)
df.index = pd.DatetimeIndex(df.index)
#print(df.index)

mpf.plot(df.iloc[72:233], type='candle')

In [ ]:
spy[72:233]